In [1]:
ls

drive/  sample_data/


In [2]:
!pip install SimpleITK

In [3]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from data import *
from Net import *
from utils import *
import matplotlib.pyplot as plt

In [5]:
# 设置CUDA设备
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

Using cuda


In [6]:
# !unzip '../MRIsource/unzipFile/6_unzip'

In [7]:
# !mv 6_unzip/* ../MRIsource/unzipFile

In [8]:
# filepath = r'../MRIsource/unzipFile'
# imagepath = r'../MRIsource/imgs'
# maskpath = r'../MRIsource/masks'
# sort_nii_image(filepath, imagepath, maskpath)

In [9]:
# 设置数据路径
dataPath = r'../MRIsource'
# 创建数据集
trainDataset = ImageDataSet(dataPath)
# # 创建数据加载器
# trainLoader = DataLoader(trainDataset, batch_size=1, shuffle=True)

# for i in range(len(trainDataset)):
#     image, segmentImage = trainDataset[i]
#     print("Image size:", image.size())
#     print("Segment Image size:", segmentImage.size())

#     # Check if the values are the same
#     if torch.equal(image, segmentImage):
#         print("Image and segmentImage values are the same")
#     else:
#         print("Image and segmentImage values are different")

In [10]:
from torch.utils.data import random_split
# 假设你有一个完整的数据集 dataset 和你想要的验证集大小 validation_size
validation_size = 3  # 根据你的需求设置验证集大小
train_size = len(trainDataset) - validation_size
# 使用 random_split 函数分割数据集
train_dataset, validation_dataset = random_split(trainDataset, [train_size, validation_size])

# 创建 DataLoader 对象
trainLoader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validationLoader = DataLoader(validation_dataset, batch_size=1, shuffle=False)

In [11]:
# 获取trainDataset中的数据数量
dataset_size = len(trainDataset)
print(f'Train dataset size: {dataset_size}')

# 获取trainLoader中的批次数量
batch_size = len(trainLoader)
print(f'Train loader batch size: {batch_size}')

# for data in trainLoader:
#     image, segmentImage = data
#     print(len(data))
#     print("Image size:", image.size())
#     print("Segment Image size:", segmentImage.size())

#     # Check if the values are the same
#     if torch.equal(image, segmentImage):
#         print("Image and segmentImage values are the same")
#     else:
#         print("Image and segmentImage values are different")



Train dataset size: 14
Train loader batch size: 11


In [12]:
from torch.cuda.amp import GradScaler, autocast
# 创建网络模型
net = UNet().to(device)

# # 初始化GradScaler
# scaler = GradScaler()
# # 将模型转换为半精度并发送到设备
# net = UNet().to(device)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.MSELoss()
#criterion =nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 用于保存最低损失
best_loss = float('inf')


In [13]:
# for i, data in enumerate(trainLoader):
#   inputs, labels = data
#   print(inputs.shape)
#   print(inputs.name)
#   print(labels.shape)
#   print(labels.name)

# for i in range(len(trainDataset)):
#     print(trainDataset.name[i])

In [14]:
import pdb
import nibabel as nib

In [15]:
import torch

def calculate_dice_score(predictions, ground_truth):
    """
    计算二分类问题的Dice系数
    predictions: 模型预测的分割图，形状为[N, H, W, D]，其中N是批次大小，H、W、D是图像的高、宽、深
    ground_truth: 实际的分割图，形状与predictions相同
    """
    # 将预测转换为二值图像
    pred = predictions > 0.5
    # 将真实标签转换为二值图像
    gt = ground_truth == 1  # 假设标签1是要分割的区域

    # 计算Dice系数
    intersection = (pred * gt).sum(dim=[1, 2, 3])  # 计算交集
    union = pred.sum(dim=[1, 2, 3]) + gt.sum(dim=[1, 2, 3])  # 计算并集
    dice = (2. * intersection) / union  # 计算Dice系数

    return dice.mean().item()  # 返回所有样本的平均Dice系数

In [17]:
#pdb.set_trace()
# 训练网络

weightPath = r'../MRIsource/params/best_model_epoch_274.pth'
net.load_state_dict(torch.load(weightPath, map_location=torch.device(device)))

validate_epochs = [1, 5, 10, 20, 100, 200, 300]
for epoch in range(2000):  # 根据您的需求设置训练的轮数
    net.to(device)
    epoch_loss=0
    net.train()  # 确保网络处于训练模式
    optimizer.zero_grad()
    for i, data in enumerate(trainLoader):
        inputs, labels = data
        #inputs, labels = inputs.half(), labels.half()  # 将输入和标签转换为半精度
        inputs, labels = inputs.to(device), labels.to(device)
        # 将labels中所有非零的元素都设置为1，注意这里使用了半精度
        labels = torch.where(labels != 0, torch.tensor(254, device=device), labels)


        # # 将labels中所有非零的元素都设置为1
        # labels = torch.where(labels != 0, torch.tensor(254, device=device), labels)
        # # print(labels.size())
        # # # 将inputs和labels转换为NumPy数组并保存为.nii文件
        # # inputs_np = inputs.cpu().numpy().squeeze(0).squeeze(0)
        # # labels_np = labels.cpu().numpy().squeeze(0).squeeze(0)

        # # # 创建nifti图像
        # # inputs_nii = nib.Nifti1Image(inputs_np, np.eye(4))
        # # labels_nii = nib.Nifti1Image(labels_np, np.eye(4))

        # # # 保存nifti图像到指定目录
        # # nib.save(inputs_nii, os.path.join('../MRIsource/label_inputs', f'inputs_epoch_{epoch+1}_batch_{i+1}.nii'))
        # # nib.save(labels_nii, os.path.join('../MRIsource/label_inputs', f'labels_epoch_{epoch+1}_batch_{i+1}.nii'))
        # # break

        # 前向传播
        outputs = net(inputs)

        #labels = labels.permute(0,1, 4, 2, 3)
        # 计算损失
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        # 反向传播和优化
        loss.backward()
        optimizer.step()
        # optimizer.zero_grad()

        print(f'Epoch {epoch+1}/{2000}, Step {i+1}/{len(trainLoader)}, Loss: {loss.item()}')

        # # 使用autocast上下文管理器包装你的前向传播过程
        # with autocast():

        #     # 前向传播
        #     outputs = net(inputs)
        #     # 计算损失，注意这里使用了半精度的inputs和labels
        #     loss = criterion(outputs, labels)

        # # 使用scaler.scale()放大损失
        # loss = scaler.scale(loss)
        # loss.backward()
        # epoch_loss += loss.item()
        # # 反向传播和优化
        # scaler.unscale_(optimizer)
        # optimizer.step()
        # scaler.update()
        # optimizer.zero_grad(set_to_none=True)  # PyTorch 1.6 以上版本使用 set_to_none=True

        # optimizer.zero_grad()  # 在开始梯度累积之前，先清零梯度
        # print(f'Epoch {epoch+1}/{200}, Step {i+1}/{len(trainLoader)}, Loss: {loss.item()}')


    # 计算平均损失
    avg_loss = epoch_loss / len(trainLoader)

    # torch.cuda.empty_cache()
    # 检查是否需要进行验证
    if epoch+1 in validate_epochs:
        net.eval()  # 确保网络处于评估模式
        dice_scores = []
        net.to('cpu')  # 将模型移动到CPU
        with torch.no_grad():  # 不需要计算梯度
            validation_loss = 0
            for i, data in enumerate(validationLoader):
                inputs, labels = data
                inputs, labels = inputs.to('cpu'), labels.to('cpu')  # 确保数据在CPU上
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                validation_loss += loss.item()
                #1
                dice_scores.append(calculate_dice_score(outputs, labels))

        # 计算平均Dice系数
        avg_dice_score = sum(dice_scores) / len(dice_scores)

        # 计算平均验证损失
        avg_validation_loss = validation_loss / len(validationLoader)
        print(f'Epoch {epoch+1}/{2000}, Train Loss: {avg_loss}, Validation Loss: {avg_validation_loss}, Test Dice Score: {avg_dice_score:.4f}')
    else:
        # 非验证 epoch
        print(f'Epoch {epoch+1}/{2000}, Train Loss: {avg_loss}, Validation Skipped.')

    # 如果平均损失更低，保存模型
    if avg_loss < best_loss:
        best_loss = avg_loss
    torch.save(net.state_dict(), f'../MRIsource/params/best_model_epoch_{epoch+1}.pth')

print('Finished Training')

Epoch 1/2000, Step 1/11, Loss: 0.5920960903167725
Epoch 1/2000, Step 2/11, Loss: 0.6225766539573669
Epoch 1/2000, Step 3/11, Loss: 0.5888165831565857
Epoch 1/2000, Step 4/11, Loss: 0.5695239305496216
Epoch 1/2000, Step 5/11, Loss: 0.5712668895721436
Epoch 1/2000, Step 6/11, Loss: 0.5514335036277771
Epoch 1/2000, Step 7/11, Loss: 0.5690769553184509
Epoch 1/2000, Step 8/11, Loss: 0.638098955154419
Epoch 1/2000, Step 9/11, Loss: 0.6241744160652161
Epoch 1/2000, Step 10/11, Loss: 0.5526989698410034
Epoch 1/2000, Step 11/11, Loss: 0.7115219831466675
Epoch 1/2000, Train Loss: 0.5992077209732749, Validation Loss: 0.6931502819061279, Test Dice Score: nan
Epoch 2/2000, Step 1/11, Loss: 0.7130206823348999
Epoch 2/2000, Step 2/11, Loss: 0.6899375319480896
Epoch 2/2000, Step 3/11, Loss: 0.701564610004425
Epoch 2/2000, Step 4/11, Loss: 0.6817703247070312
Epoch 2/2000, Step 5/11, Loss: 0.6298902034759521
Epoch 2/2000, Step 6/11, Loss: 0.5818353891372681
Epoch 2/2000, Step 7/11, Loss: 0.5562314987182

KeyboardInterrupt: 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')